In [13]:
import os
import re
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
from dotenv import load_dotenv
import gc


gc.collect()
torch.cuda.empty_cache()

load_dotenv()


True

In [14]:
def read_txt(file_path):
    with open(file_path, "r") as file:
        text = file.read()
    return text

def read_documents_from_directory(directory):
    combined_text = ""
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        combined_text += read_txt(file_path)
    return combined_text


In [15]:
def train_chatbot(directory, model_output_path, train_fraction=0.8):

    combined_text = read_documents_from_directory(directory)
    combined_text = re.sub(r'\n+', '\n', combined_text).strip() 

    split_index = int(train_fraction * len(combined_text))
    train_text = combined_text[:split_index]
    val_text = combined_text[split_index:]

    with open("train.txt", "w") as f:
        f.write(train_text)
    with open("val.txt", "w") as f:
        f.write(val_text)

    tokenizer = GPT2Tokenizer.from_pretrained("gpt2-medium")
    model = GPT2LMHeadModel.from_pretrained("gpt2-medium")

    
    train_dataset = TextDataset(tokenizer=tokenizer, file_path="train.txt", block_size=128)
    val_dataset = TextDataset(tokenizer=tokenizer, file_path="val.txt", block_size=128)
    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

    training_args = TrainingArguments(
        output_dir=model_output_path,
        overwrite_output_dir=True,
        per_device_train_batch_size=1, 
        per_device_eval_batch_size=1,
        num_train_epochs=100,
        save_steps=10_000,
        save_total_limit=2,
        logging_dir='./logs',
        hub_strategy="checkpoint",
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
    )

    trainer.train(resume_from_checkpoint=True)
    trainer.save_model(model_output_path)
    
    tokenizer.save_pretrained(model_output_path)


In [16]:
def generate_response(model, tokenizer, prompt, max_length=100):
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    
    attention_mask = torch.ones_like(input_ids)
    pad_token_id = tokenizer.eos_token_id

    output = model.generate(
        input_ids,
        max_length=max_length,
        num_return_sequences=1,
        attention_mask=attention_mask,
        pad_token_id=pad_token_id
    )

    return tokenizer.decode(output[0], skip_special_tokens=True)


In [17]:
def main():
    project_directory = os.getenv("PROJECT_DIR")
    data_directory = os.path.join(project_directory, r"Data/cleaned_data")
    model_output_path = os.path.join(project_directory, r"Models/")

    train_chatbot(data_directory, model_output_path)

    model = GPT2LMHeadModel.from_pretrained(model_output_path)
    tokenizer = GPT2Tokenizer.from_pretrained(model_output_path)

    prompt = "What is Canvas?"  
    response = generate_response(model, tokenizer, prompt)
    print("Generated response:", response)

In [18]:
if __name__ == "__main__":
    main()

/home/chuongp100/yes/lib/python3.11/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/chuongp100/yes/lib/python3.11/site-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
/home/chuongp100/yes/lib/python3.11/site-packages/accelerate/accelerator.py:451: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead:

Step,Training Loss
240500,0.029900
241000,0.025600
241500,0.027600
242000,0.026100
242500,0.026700
243000,0.027800
243500,0.029000
244000,0.029200
244500,0.029600
245000,0.024200


/home/chuongp100/yes/lib/python3.11/site-packages/transformers/modeling_utils.py:479: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkpoint_file, map_lo

Generated response: What is Canvas?
Canvas is a Learning Management System. To learn more about Canvas terminology and definitions, visit How does Canvas define the terms used to describe its features and functions?
Because Canvas is a web-based system, it doesn’t need to be installed on your computer. However, you’ll want to make sure that your computer and web browser meet the basic requirements to run Canvas.
What is the Instructor/Teacher role?



In [2]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import os
from dotenv import load_dotenv

load_dotenv()

/home/chuongp100/yes/lib/python3.11/site-packages/transformers/utils/generic.py:260: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
/home/chuongp100/yes/lib/python3.11/site-packages/transformers/utils/generic.py:260: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(


True

In [3]:
def generate_response(model, tokenizer, prompt, max_length=250):
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    
    attention_mask = torch.ones_like(input_ids)
    pad_token_id = tokenizer.eos_token_id

    output = model.generate(
        input_ids,
        max_length=max_length,
        num_return_sequences=1,
        attention_mask=attention_mask,
        pad_token_id=pad_token_id
    )

    return tokenizer.decode(output[0], skip_special_tokens=True)


In [4]:
project_directory = os.getenv("PROJECT_DIR")
model_path = os.path.join(project_directory, r"Models/")

my_chat_model = GPT2LMHeadModel.from_pretrained(model_path)
my_chat_tokenizer = GPT2Tokenizer.from_pretrained(model_path)

/home/chuongp100/yes/lib/python3.11/site-packages/transformers/modeling_utils.py:479: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkpoint_file, map_lo

In [6]:
prompt = "Can students view unpublished assignments?"  

response = generate_response(my_chat_model, my_chat_tokenizer, prompt, max_length=1024)  
print("Generated response:", response)

Generated response: Can students view unpublished assignments?
Yes. Students can view unpublished assignments on the Assignments page. Learn more about publishing assignments.
Unpublished assignments are identified by the Unpublished icon. You can view the assignment name, but students cannot view the assignment.
Learn more about publishing assignments. 
Review Assignment Details
Students can view details about an assignment. When an assignment is created, the assignment details display. 
Depending on the assignment submission type, students may be able to view various options for the assignment, including the assignment description.
Students may also be able to view details about the assignment using the Edit button.
View Feedback
If your assignment has been affected by a feedback from your instructor, students may be able to view that feedback in their Grades page.
You can view the feedback in the Grades page by clicking the View Feedback link. How do I archive a grading scheme in an